# TODO：概要

# ライブラリのインポート

In [17]:
import os
import time
import json
import datetime
import zoneinfo

import requests

from dotenv import load_dotenv, find_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import (
    MessageTextContent,
    ListSortOrder,
    McpTool,
    MCPToolDefinition,
    RequiredMcpToolCall,
    SubmitToolApprovalAction,
    ToolApproval,
    CodeInterpreterTool,
    FunctionTool,
    ToolSet,
    FilePurpose,
    FileSearchTool,
    RunAdditionalFieldList,
    RunStepFileSearchToolCall,
    RunStepToolCallDetails,
)


# 環境変数の取得

In [18]:
load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")

# クライアントの初期化

In [19]:
# AI Project Client を初期化
project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

# AgentClient の作成
agents_client = project_client.agents

# ユーティリティ関数

In [20]:
def agent_run_outputs(thread_id, agents_client, target_dir="./output_images"):
    """
    指定したスレッドIDのRun実行結果（テキスト・画像）をNotebook上に表示＆画像は保存。
    """
    messages = agents_client.messages.list(thread_id=thread_id, order=ListSortOrder.ASCENDING)
    os.makedirs(target_dir, exist_ok=True)

    for message in messages:
        # テキスト出力
        if message.text_messages:
            for txt in message.text_messages:
                print(f"{message.role.upper()}: {txt.text.value}")
        
        # 画像出力
        if hasattr(message, "image_contents"):
            for image_content in message.image_contents:
                file_id = image_content.image_file.file_id
                file_name = f"{file_id}_image_file.png"

                agents_client.files.save(
                    file_id=file_id,
                    file_name=file_name,
                    target_dir=target_dir
                )
                print(f"Saved image: {file_name}")
                display(Image(filename=f"{target_dir}/{file_name}"))

In [21]:
def file_search_content_outputs(agents_client, thread_id, run_id):
    """
    指定したスレッドID・Run IDのファイル検索ツールによる検索結果コンテンツをNotebook等に出力します。

    :param agents_client: AgentClient インスタンス
    :param thread_id: スレッドID
    :param run_id: Run ID
    """
    from azure.ai.agents.models import RunAdditionalFieldList, RunStepToolCallDetails, RunStepFileSearchToolCall

    for run_step in agents_client.run_steps.list(
        thread_id=thread_id,
        run_id=run_id,
        include=[RunAdditionalFieldList.FILE_SEARCH_CONTENTS]
    ):
        if isinstance(run_step.step_details, RunStepToolCallDetails):
            for tool_call in run_step.step_details.tool_calls:
                if (
                    isinstance(tool_call, RunStepFileSearchToolCall)
                    and getattr(tool_call, "file_search", None)
                    and getattr(tool_call.file_search, "results", None)
                    and tool_call.file_search.results
                    and getattr(tool_call.file_search.results[0], "content", None)
                    and tool_call.file_search.results[0].content
                    and getattr(tool_call.file_search.results[0].content[0], "text", None)
                ):
                    file_name = tool_call.file_search.results[0].file_name
                    content_text = tool_call.file_search.results[0].content[0].text
                    print(f"ファイル検索の結果、以下のコンテンツがヒットしました。\n「{file_name}」:")
                    print("=" * 60)
                    print(content_text)
                    print("=" * 60)

# ツールの定義

## ベクトルストアの作成

In [22]:
# アップロード対象ディレクトリ
input_dir = "./input_data"

# ファイルIDのリスト
file_ids = []

# ディレクトリ内の全ファイルをループしてアップロード
for fname in os.listdir(input_dir):
    file_path = os.path.join(input_dir, fname)
    if os.path.isfile(file_path):
        uploaded_file = agents_client.files.upload_and_poll(file_path=file_path, purpose=FilePurpose.AGENTS)
        print(f"Uploaded file: {fname}, file ID: {uploaded_file.id}")
        file_ids.append(uploaded_file.id)

if not file_ids:
    raise RuntimeError("No files were uploaded. Please check the directory.")

# アップロードした全ファイルを使ってベクトルストアを作成
vector_store = agents_client.vector_stores.create_and_poll(file_ids=file_ids, name="my_vectorstore")
print(f"Created vector store, vector store ID: {vector_store.id}")


Uploaded file: カスタマーサービスのパンフレット.pptx, file ID: assistant-GsfetxxiaDLNwe1TaPQGQ1
Uploaded file: 出荷日ポリシー.pdf, file ID: assistant-XhmLSe9mLyUUHAFshHwvjh
Created vector store, vector store ID: vs_kqDSa9H5OEOZTlHV6yGBZlFR


In [23]:
# インポートしたファイル数の確認
vector_store.as_dict()
vector_store.file_counts

{'in_progress': 0, 'completed': 2, 'failed': 0, 'cancelled': 0, 'total': 2}

## File Search ツールの定義

In [24]:
# Code Interpreter ツールの定義
file_search = FileSearchTool(vector_store_ids=[vector_store.id])

# エージェントの作成

In [25]:
file_search_agent  = agents_client.create_agent(
    model=AZURE_DEPLOYMENT_NAME,
    name="file_search_agent ",
    instructions=(
        "アップロードしたファイルから情報を検索できるエージェントです。"
        "ユーザーの質問に、アップロードしたファイルを参照して答えてください。"
    ),
    description=(
        "アップロードしたファイルから情報を検索できるエージェントです。"    
    ),
    tools=file_search.definitions,  
    tool_resources=file_search.resources,
    temperature=0.1
)
print(f"Created Agent. AGENT_ID: {file_search_agent.id}")


Created Agent. AGENT_ID: asst_pZkH18sbpg4K9VN7GssuZbLS


In [33]:
agent_dict = file_search_agent.as_dict()
print(json.dumps(agent_dict, indent=2, ensure_ascii=False))

{
  "id": "asst_pZkH18sbpg4K9VN7GssuZbLS",
  "object": "assistant",
  "created_at": 1753562863,
  "name": "file_search_agent ",
  "description": "アップロードしたファイルから情報を検索できるエージェントです。",
  "model": "gpt-4.1",
  "instructions": "アップロードしたファイルから情報を検索できるエージェントです。ユーザーの質問に、アップロードしたファイルを参照して答えてください。",
  "tools": [
    {
      "type": "file_search"
    }
  ],
  "top_p": 1.0,
  "temperature": 0.1,
  "tool_resources": {
    "file_search": {
      "vector_store_ids": [
        "vs_kqDSa9H5OEOZTlHV6yGBZlFR"
      ]
    }
  },
  "metadata": {},
  "response_format": "auto"
}


# スレッドの作成

In [26]:
# Thread の作成
thread = agents_client.threads.create()
print(f"Created Thread. THREAD_ID: {thread.id}")

Created Thread. THREAD_ID: thread_Usj6oblNygKMQvC0d3X4aU25


# ユーザーメッセージの追加

In [27]:
# メッセージの追加
user_message_1 = "新しいパソコンの購入を検討しています。しかし、初期設定に自信がないので、初期設定サポートについて教えてください。"

message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message_1,
)

print(f"Added Message. MESSAGE_ID: {message.id}")

Added Message. MESSAGE_ID: msg_aolksdQt9rmUXT3RivFDHZSE


# Run の実行

## 1 ターン目

In [28]:
run_1 = agents_client.runs.create_and_process(
    thread_id=thread.id,
    agent_id=file_search_agent.id
)

if run_1.status == "failed":
    print(f"Run failed: {run_1.last_error}")
else:
    agent_run_outputs(thread.id, agents_client)

USER: 新しいパソコンの購入を検討しています。しかし、初期設定に自信がないので、初期設定サポートについて教えてください。
ASSISTANT: アップロードされたファイルの中から「初期設定サポート」に関する情報を検索します。少々お待ちください。
ASSISTANT: パソコンの初期設定サポートについて、アップロードされた資料に以下の内容が記載されています。

---

### 🎉初心者向けセットアップサポート🎉
**～パソコン・スマホの初期設定を安心サポート～**

#### サービス内容（所要時間：約30〜60分）
- 電源の接続と起動確認
- Wi-Fi・インターネット接続設定
- メールアカウント・アプリの初期登録
- セキュリティ対策（ウイルス対策ソフトの確認・導入）
- 基本操作レクチャー（マウス、キーボード、タッチ操作）
- よく使うアプリのインストールサポート（Office・Zoomなど）

#### こんな方におすすめ
- 初めてパソコンやスマホを購入した方
- 設定がよくわからず不安な方
- 家族に頼らず自分で使いたい方

#### サポート体制
- 出張対応／オンライン対応 選べます
- 初回サポート無料（キャンペーン中）
- ご希望に応じてマニュアルもプレゼント

#### お問い合わせ
- 電話：0120-123-456（平日 10:00〜18:00）
- メール：support@easysetup.jp
- WEB予約：https://www.easysetup.jp

---

このように、初期設定のサポートは電源接続からインターネット設定、アプリのインストールまで幅広く対応しており、出張やオンラインでのサポートも選択できます。初回無料キャンペーンも実施中ですので、安心してご利用いただけます【5:0†カスタマーサービスのパンフレット.pptx】。


In [29]:
# 参照したコンテンツを出力
file_search_content_outputs(agents_client, thread.id, run_1.id)


ファイル検索の結果、以下のコンテンツがヒットしました。
「カスタマーサービスのパンフレット.pptx」:
🎉初心者向けセットアップサポート🎉 ～パソコン・スマホの初期設定を安心サポート～ 
✅ サービス内容（所要時間：約30〜60分） 電源の接続と起動確認 Wi-Fi・インターネット接続設定 メールアカウント・アプリの初期登録 セキュリティ対策（ウイルス対策ソフトの確認・導入） 基本操作レクチャー（マウス、キーボード、タッチ操作） よく使うアプリのインストールサポート（ Office ・Zoomなど） 
👩‍💻 こんな方におすすめ ！ 初めてパソコンやスマホを購入した方 設定がよくわからず不安な方 家族に頼らず自分で使いたい方 
📞 カスタマーサポート ご不明点やご予約は下記までお気軽に！ 電話：0120-123-456（平日 10:00〜18:00） メール：support@easysetup.jp WEB予約： https://www.easysetup.jp 
💡安心のサポート体制 ✔ 出張対応／オンライン対応 選べます ✔ 初回サポート無料（キャンペーン中） ✔ ご希望に応じてマニュアルもプレゼント！


## 2 ターン目

In [30]:
# メッセージの追加
user_message_2 = "PCを購入した際、どれくらいで出荷されますか？"

message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message_2,
)

run_2 = agents_client.runs.create_and_process(
    thread_id=thread.id,
    agent_id=file_search_agent.id
)

if run_2.status == "failed":
    print(f"Run failed: {run_2.last_error}")
else:
    agent_run_outputs(thread.id, agents_client)

USER: 新しいパソコンの購入を検討しています。しかし、初期設定に自信がないので、初期設定サポートについて教えてください。
ASSISTANT: アップロードされたファイルの中から「初期設定サポート」に関する情報を検索します。少々お待ちください。
ASSISTANT: パソコンの初期設定サポートについて、アップロードされた資料に以下の内容が記載されています。

---

### 🎉初心者向けセットアップサポート🎉
**～パソコン・スマホの初期設定を安心サポート～**

#### サービス内容（所要時間：約30〜60分）
- 電源の接続と起動確認
- Wi-Fi・インターネット接続設定
- メールアカウント・アプリの初期登録
- セキュリティ対策（ウイルス対策ソフトの確認・導入）
- 基本操作レクチャー（マウス、キーボード、タッチ操作）
- よく使うアプリのインストールサポート（Office・Zoomなど）

#### こんな方におすすめ
- 初めてパソコンやスマホを購入した方
- 設定がよくわからず不安な方
- 家族に頼らず自分で使いたい方

#### サポート体制
- 出張対応／オンライン対応 選べます
- 初回サポート無料（キャンペーン中）
- ご希望に応じてマニュアルもプレゼント

#### お問い合わせ
- 電話：0120-123-456（平日 10:00〜18:00）
- メール：support@easysetup.jp
- WEB予約：https://www.easysetup.jp

---

このように、初期設定のサポートは電源接続からインターネット設定、アプリのインストールまで幅広く対応しており、出張やオンラインでのサポートも選択できます。初回無料キャンペーンも実施中ですので、安心してご利用いただけます【5:0†カスタマーサービスのパンフレット.pptx】。
USER: PCを購入した際、どれくらいで出荷されますか？
ASSISTANT: PC（パソコン）を購入した場合の出荷日は、「受注後5〜7営業日以内」となっています。ただし、設置工事が必要な商品やメーカーから取り寄せが必要な場合は、さらに日数がかかることがあります。また、営業日は土日祝日を除く平日ですのでご注意ください【9:0†出荷日ポリシー.pdf】。


In [31]:
# 参照したコンテンツを出力
file_search_content_outputs(agents_client, thread.id, run_2.id)

ファイル検索の結果、以下のコンテンツがヒットしました。
「出荷日ポリシー.pdf」:
2025-07-26

1 / 2

出荷⽇ポリシー
当 EC サイトでは、お客様に安⼼してショッピングをお楽しみいただくため、商品のカテゴリに応じた出荷
⽇ポリシーを設けております。ご注⽂いただいた商品は、以下のポリシーに基づき順次発送されます。

1. デジタルコンテンツ・ダウンロード商品

出荷⽇︓ご購⼊後、即時ダウンロード可能
詳細︓決済完了と同時に、ダウンロードリンクが記載されたメールが⾃動送信されます。物理的な商
品の発送はありません。

2. 書籍・CD/DVD・ゲームソフト

出荷⽇︓受注後2〜3営業⽇以内
詳細︓在庫がある場合、注⽂直後の変更受付は出荷準備に移るため不可。新刊・予約商品は、発売⽇
以降の出荷となります。

3. ファッション・アパレル・雑貨

出荷⽇︓受注後3〜5営業⽇以内
詳細︓在庫状況や⽣産状況により、出荷までにお時間をいただく場合がございます。発送予定⽇や⼊
荷情報は、別途記載の納期欄でご確認ください。

※注意点︓アパレル・雑貨の中でもキャラクターグッズに関しては、ここに記載の通りではございません。
別途「キャラクターグッズ出荷⽇ポリシー」をご確認ください。

4. PC・電化製品・家具・⼤型商品

出荷⽇︓受注後5〜7営業⽇以内
詳細︓設置⼯事が必要な商品や、メーカーから取り寄せが必要な商品は、上記以上の納期をいただく
場合がございます。別途、配送業者からの連絡が⼊る場合がございます。

5. ⾷品・飲料（常温品）


# Agent ID を .env ファイルに保存
※ 今回作成したエージェントを、後続の Connected Agents のハンズオン演習で使用するため永続化します。

In [32]:
# 変数の定義
agent_env_key = "FOUNDRY_FILE_SEARCH_AGENT_ID"
agent_env_value = file_search_agent.id

# .envファイルのパスを自動探索
env_path = find_dotenv()  # 見つからなければ''を返す
if not env_path:
    raise FileNotFoundError(".envファイルが見つかりませんでした。")

# AGENT_ID を .env ファイルに追記
with open(env_path, "a", encoding="utf-8") as f:
    f.write(f'\n{agent_env_key}="{agent_env_value}"')

print(f'.envファイルに {agent_env_key}=\"{agent_env_value}\" を追記しました。')


.envファイルに FOUNDRY_FILE_SEARCH_AGENT_ID="asst_pZkH18sbpg4K9VN7GssuZbLS" を追記しました。
